In [ ]:
!pip install arabert transformers datasets accelerate>=0.20.1 -q

In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

In [ ]:
train_df = pd.read_csv('/content/train_df_combined_with_embedding.csv')

In [ ]:
train_df.head(3)

,level_0,index,Q_Answers,Question,Answer,Embeddings
0,0,0,7:85-93,من هم قوم شعيب؟,وإلى مدين أخاهم شعيبا قال يا قوم اعبدوا الله م...,"[0.0436842106282711, -0.0241576936095953, -0.0..."
1,1,1,11:84-88,من هم قوم شعيب؟,وإلى مدين أخاهم شعيبا قال يا قوم اعبدوا الله م...,"[0.0436842106282711, -0.0241576936095953, -0.0..."
2,2,2,11:84-88,ما هي اسماء المدن المذكورة في القرآن؟,وإلى مدين أخاهم شعيبا قال يا قوم اعبدوا الله م...,"[0.09956597536802292, -0.049366891384124756, -..."


In [ ]:
def average_pool(last_hidden_states, attention_mask):
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-small')

In [ ]:
matrix_of_embeds = Tensor([eval(x) for x in (train_df['Embeddings'].tolist())])

In [ ]:
custom_question = 'query: ' + 'من هم الحيوانات؟'

batch_dict_custom = tokenizer([custom_question], max_length=128, padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
  outputs_custom = model(**batch_dict_custom)

embeddings_custom = average_pool(outputs_custom.last_hidden_state, batch_dict_custom['attention_mask'])

embeddings_custom = F.normalize(embeddings_custom, p=2, dim=1)

scores = ((embeddings_custom @ matrix_of_embeds.T) * 100)[0]

In [ ]:
train_df['score'] = scores

In [ ]:
train_df.iloc[scores.sort(descending=True).indices[:10]]

,level_0,index,Q_Answers,Question,Answer,Embeddings,score
350,350,350,20:80-82,ما هي انواع الحيوانات التي ذكرت في القرآن؟,يا بني إسرائيل قد أنجيناكم من عدوكم وواعدناكم ...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
987,987,66,7:163-166,ما هي انواع الحيوانات التي ذكرت في القرآن؟,واسألهم عن القرية التي كانت حاضرة البحر إذ يعد...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
364,364,364,27:82-88,ما هي انواع الحيوانات التي ذكرت في القرآن؟,وإذا وقع القول عليهم أخرجنا لهم دابة من الأرض ...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
361,361,361,27:20-28,ما هي انواع الحيوانات التي ذكرت في القرآن؟,وتفقد الطير فقال ما لي لا أرى الهدهد أم كان من...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
360,360,360,26:141-159,ما هي انواع الحيوانات التي ذكرت في القرآن؟,كذبت ثمود المرسلين. إذ قال لهم أخوهم صالح ألا ...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
359,359,359,26:23-40,ما هي انواع الحيوانات التي ذكرت في القرآن؟,قال فرعون وما رب العالمين. قال رب السماوات وال...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
358,358,358,22:73-76,ما هي انواع الحيوانات التي ذكرت في القرآن؟,يا أيها الناس ضرب مثل فاستمعوا له إن الذين تدع...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
354,354,354,22:30-37,ما هي انواع الحيوانات التي ذكرت في القرآن؟,ذلك ومن يعظم حرمات الله فهو خير له عند ربه وأح...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
352,352,352,21:87-88,ما هي انواع الحيوانات التي ذكرت في القرآن؟,وذا النون إذ ذهب مغاضبا فظن أن لن نقدر عليه فن...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
988,988,94,27:15-22,ما هي انواع الحيوانات التي ذكرت في القرآن؟,ولقد آتينا داوود وسليمان علما وقالا الحمد لله ...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109


In [ ]:
train_df.iloc[scores.sort(descending=True).indices[:]]

,level_0,index,Q_Answers,Question,Answer,Embeddings,score
350,350,350,20:80-82,ما هي انواع الحيوانات التي ذكرت في القرآن؟,يا بني إسرائيل قد أنجيناكم من عدوكم وواعدناكم ...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
987,987,66,7:163-166,ما هي انواع الحيوانات التي ذكرت في القرآن؟,واسألهم عن القرية التي كانت حاضرة البحر إذ يعد...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
364,364,364,27:82-88,ما هي انواع الحيوانات التي ذكرت في القرآن؟,وإذا وقع القول عليهم أخرجنا لهم دابة من الأرض ...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
361,361,361,27:20-28,ما هي انواع الحيوانات التي ذكرت في القرآن؟,وتفقد الطير فقال ما لي لا أرى الهدهد أم كان من...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
360,360,360,26:141-159,ما هي انواع الحيوانات التي ذكرت في القرآن؟,كذبت ثمود المرسلين. إذ قال لهم أخوهم صالح ألا ...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
...,...,...,...,...,...,...,...
957,957,957,-1,أين نزلت سورة العلق؟,لا جواب,"[0.08692284673452377, -0.053114037960767746, -...",71.862190
420,420,420,12:50-57,كيف خرج سيدنا يوسف عليه السلام من السجن؟,وقال الملك ائتوني به فلما جاءه الرسول قال ارجع...,"[0.10006450116634369, -0.04097944498062134, -0...",70.962402
1158,1158,627,12:39-54,كيف خرج سيدنا يوسف عليه السلام من السجن؟,يا صاحبي السجن أأرباب متفرقون خير أم الله الوا...,"[0.10006450116634369, -0.04097944498062134, -0...",70.962402
300,300,300,10:87-89,هل عاش سيدنا موسى في مصر؟,وأوحينا إلى موسى وأخيه أن تبوآ لقومكما بمصر بي...,"[0.06306566298007965, -0.062337927520275116, 0...",70.167816


In [ ]:
df_run = train_df.drop(['level_0'], axis = 1)

In [ ]:
df_run.iloc[scores.sort(descending=True).indices[:]]

,index,Q_Answers,Question,Answer,Embeddings,score
350,350,20:80-82,ما هي انواع الحيوانات التي ذكرت في القرآن؟,يا بني إسرائيل قد أنجيناكم من عدوكم وواعدناكم ...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
987,66,7:163-166,ما هي انواع الحيوانات التي ذكرت في القرآن؟,واسألهم عن القرية التي كانت حاضرة البحر إذ يعد...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
364,364,27:82-88,ما هي انواع الحيوانات التي ذكرت في القرآن؟,وإذا وقع القول عليهم أخرجنا لهم دابة من الأرض ...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
361,361,27:20-28,ما هي انواع الحيوانات التي ذكرت في القرآن؟,وتفقد الطير فقال ما لي لا أرى الهدهد أم كان من...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
360,360,26:141-159,ما هي انواع الحيوانات التي ذكرت في القرآن؟,كذبت ثمود المرسلين. إذ قال لهم أخوهم صالح ألا ...,"[0.07337567210197449, -0.03972385823726654, -0...",91.649109
...,...,...,...,...,...,...
957,957,-1,أين نزلت سورة العلق؟,لا جواب,"[0.08692284673452377, -0.053114037960767746, -...",71.862190
420,420,12:50-57,كيف خرج سيدنا يوسف عليه السلام من السجن؟,وقال الملك ائتوني به فلما جاءه الرسول قال ارجع...,"[0.10006450116634369, -0.04097944498062134, -0...",70.962402
1158,627,12:39-54,كيف خرج سيدنا يوسف عليه السلام من السجن؟,يا صاحبي السجن أأرباب متفرقون خير أم الله الوا...,"[0.10006450116634369, -0.04097944498062134, -0...",70.962402
300,300,10:87-89,هل عاش سيدنا موسى في مصر؟,وأوحينا إلى موسى وأخيه أن تبوآ لقومكما بمصر بي...,"[0.06306566298007965, -0.062337927520275116, 0...",70.167816


In [ ]:
df_run['rank'] =  df_run.groupby('Question')['score'].rank(method='first').astype(int)

In [ ]:
# To Identify unique question in the column
unique_question = df_run['Question'].unique()

# To Create a mapping of unique question to categorical labels
mapping = {Question: f'{i+1}' for i, Question in enumerate(unique_question)}

# To Replace the original question with their corresponding labels
df_run['question-id'] = df_run['Question'].map(mapping)

In [ ]:
# 4. save the run in trec format to a file
df_run["Q0"] = ["Q0"] * len(df_run)
df_run["tag"] = ["PUST_run1.tsv"] * len(df_run)
#df_run['question-id'] = df_run["index"]
df_run['passage-id'] = df_run["Q_Answers"]
df_run = df_run[["question-id", "Q0", "passage-id", "rank", "score", "tag"]]
df_run.to_csv('PUST_run1.tsv', sep="\t", index=False, header=False)
df_run

,question-id,Q0,passage-id,rank,score,tag
0,1,Q0,7:85-93,1,77.512306,PUST_run1.tsv
1,1,Q0,11:84-88,2,77.512306,PUST_run1.tsv
2,2,Q0,11:84-88,1,77.194145,PUST_run1.tsv
3,1,Q0,26:176-191,3,77.512306,PUST_run1.tsv
4,1,Q0,29:36-37,4,77.512306,PUST_run1.tsv
...,...,...,...,...,...,...
1179,39,Q0,27:15-22,2,74.066589,PUST_run1.tsv
1180,190,Q0,28:76-81,1,80.218018,PUST_run1.tsv
1181,9,Q0,28:76-81,4,76.842857,PUST_run1.tsv
1182,57,Q0,83:1-17,2,80.255112,PUST_run1.tsv
